# To do
* split
* check dtypes
* convert to parquet
* start to play

# Machine info

In [1]:
!cat /proc/cpuinfo | grep -m1 "model name"
!awk '$3=="kB"{$2=$2/1024^2;$3="GB";} 1' /proc/meminfo | column -t | grep "MemTotal"

model name	: Intel(R) Core(TM) i3-4000M CPU @ 2.40GHz
MemTotal:           15.3896      GB


# Yellow Taxi Trip Data

* [2017](https://data.cityofnewyork.us/Transportation/2017-Yellow-Taxi-Trip-Data/biws-g3hs)
* [2018](https://data.cityofnewyork.us/Transportation/2018-Yellow-Taxi-Trip-Data/t29m-gskq)
* Full months by months [link](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

In [2]:
!ls -lh datasets/nyctaxi/raw/

total 20G
-rw-rw-r-- 1 baobob baobob 9.9G Apr  4  2019 2017_Yellow_Taxi_Trip_Data.csv
-rw-rw-r-- 1 baobob baobob 9.8G Feb 28 11:05 2018_Yellow_Taxi_Trip_Data.csv


In [1]:
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client, LocalCluster
client = Client()
client

Client Scheduler: tcp://127.0.0.1:40589 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [2]:
df = dd.read_csv("datasets/nyctaxi/raw/*.csv")

In [3]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
npartitions=328,,,,,,,,,,,,,,,,,
,int64,object,object,int64,float64,int64,object,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,05/20/2017 05:41:25 PM,05/20/2017 05:58:43 PM,6,2.1,1,N,161,107,1,12.5,0.0,0.5,0.00,0.0,0.3,13.30
1,1,05/20/2017 05:41:26 PM,05/20/2017 05:51:26 PM,1,1.1,1,N,48,186,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80
2,1,05/20/2017 05:41:26 PM,05/20/2017 06:00:03 PM,1,1.4,1,N,48,161,1,12.0,0.0,0.5,2.55,0.0,0.3,15.35
3,2,05/20/2017 05:41:26 PM,05/20/2017 05:53:11 PM,2,1.6,1,N,237,263,1,10.0,0.0,0.5,2.70,0.0,0.3,13.50
4,2,05/20/2017 05:41:26 PM,05/20/2017 05:46:17 PM,2,1.1,1,N,140,262,1,6.0,0.0,0.5,0.00,0.0,0.3,6.80


In [5]:
%%time
out = df.groupby(df.passenger_count).trip_distance.mean().compute()

CPU times: user 1min 33s, sys: 9.33 s, total: 1min 42s
Wall time: 14min 26s


In [6]:
out

passenger_count
0      2.762565
1      2.877378
2      3.091487
3      3.041540
4      3.111632
5      3.012087
6      2.990773
7      3.275516
8      5.036860
9      5.982670
192    1.010000
96     0.830000
Name: trip_distance, dtype: float64

In [7]:
out.reset_index(name="avg_distance")

,passenger_count,avg_distance
0,0,2.762565
1,1,2.877378
2,2,3.091487
3,3,3.041540
4,4,3.111632
5,5,3.012087
6,6,2.990773
7,7,3.275516
8,8,5.036860
9,9,5.982670


# Divide et Impera

In [1]:
%%bash
for year in {2017..2018}
do
mkdir -p datasets/nyctaxi/raw_parts/year=${year}
done

In [2]:
%%time
%%bash
cat datasets/nyctaxi/raw/2017_Yellow_Taxi_Trip_Data.csv | parallel --header : --pipe -N1000000 'cat >datasets/nyctaxi/raw_parts/year=2017/file_{#}.csv'
cat datasets/nyctaxi/raw/2018_Yellow_Taxi_Trip_Data.csv | parallel --header : --pipe -N1000000 'cat >datasets/nyctaxi/raw_parts/year=2018/file_{#}.csv'

parallel: Warning: A record was longer than 1048576. Increasing to --blocksize 1363150.
parallel: Warning: A record was longer than 1363150. Increasing to --blocksize 1772096.
parallel: Warning: A record was longer than 1772096. Increasing to --blocksize 2303726.
parallel: Warning: A record was longer than 2303726. Increasing to --blocksize 2994845.
parallel: Warning: A record was longer than 2994845. Increasing to --blocksize 3893300.
parallel: Warning: A record was longer than 3893300. Increasing to --blocksize 5061291.
parallel: Warning: A record was longer than 5061291. Increasing to --blocksize 6579680.
parallel: Warning: A record was longer than 6579680. Increasing to --blocksize 8553585.
parallel: Warning: A record was longer than 8553585. Increasing to --blocksize 11119662.
parallel: Warning: A record was longer than 11119662. Increasing to --blocksize 14455562.
parallel: Warning: A record was longer than 14455562. Increasing to --blocksize 18792232.
parallel: Warning: A record

CPU times: user 15.7 ms, sys: 14.2 ms, total: 29.9 ms
Wall time: 7min 41s


## Convert to `.parquet`

In [1]:
import pandas as pd
import os
from dask import delayed, compute
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:35827 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [2]:
jobs = []
fldr_in = "datasets/nyctaxi/raw_parts/"
for (dirpath, dirnames, filenames) in os.walk(fldr_in):
    if len(filenames) > 0:
        jobs += [os.path.join(dirpath, fn)
                 for fn in filenames]

## Change `dtypes`

In [3]:
fn = jobs[0]

In [4]:
%%time
df = pd.read_csv(fn)

CPU times: user 2.16 s, sys: 194 ms, total: 2.36 s
Wall time: 2.24 s


In [5]:
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object

In [6]:
mem_ini = df.memory_usage(deep=True).sum()/1024**2
print(mem_ini)

320.4346923828125


In [7]:
%%time
for col in df.columns:
    if "datetime" in col:
        df[col] = pd.to_datetime(df[col],
                  format='%m/%d/%Y %I:%M:%S %p')
    if df[col].dtype == "float64":
        df[col] = df[col].astype("float16")
    if df[col].dtype == "int64":
        df[col] = df[col].astype("int16")

CPU times: user 6.3 s, sys: 222 ms, total: 6.53 s
Wall time: 6.43 s


In [8]:
mem_fin = df.memory_usage(deep=True).sum()/1024**2
mem_fin

104.904296875

In [9]:
mem_fin/mem_ini

0.3273812086166824

## Convert

In [10]:
%%bash
for year in {2017..2018}
do
mkdir -p datasets/nyctaxi/processed/year=${year}
done

In [11]:
@delayed
def conver2parquet(fn, fldr_in, fldr_out):
    fn_out = fn.replace(fldr_in, fldr_out)\
               .replace(".csv", ".parquet")
    df = pd.read_csv(fn)
    for col in df.columns:
        if "datetime" in col:
            df[col] = pd.to_datetime(df[col],
                      format='%m/%d/%Y %I:%M:%S %p')
        if df[col].dtype == "float64":
            df[col] = df[col].astype("float32")
        if df[col].dtype == "int64":
            df[col] = df[col].astype("int32")
    df.to_parquet(fn_out, index=False)


In [12]:
fldr_in = "raw_parts"
fldr_out = "processed"

In [13]:
%%time
to_process = [conver2parquet(job, fldr_in, fldr_out) for job in jobs]
out = compute(to_process)

CPU times: user 1min 28s, sys: 9.64 s, total: 1min 38s
Wall time: 21min 9s


In [11]:
%%bash
du -h datasets/nyctaxi/raw/
du -h datasets/nyctaxi/raw_parts/
du -h datasets/nyctaxi/processed/

20G	datasets/nyctaxi/raw/
9.9G	datasets/nyctaxi/raw_parts/year=2017
9.8G	datasets/nyctaxi/raw_parts/year=2018
20G	datasets/nyctaxi/raw_parts/
1.7G	datasets/nyctaxi/processed/year=2017
1.8G	datasets/nyctaxi/processed/year=2018
3.5G	datasets/nyctaxi/processed/


# Use parquets

In [1]:
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:43161 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [2]:
df = dd.read_parquet("datasets/nyctaxi/processed/")

In [3]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,year
npartitions=227,,,,,,,,,,,,,,,,,,
,int32,datetime64[ns],datetime64[ns],int32,float32,int32,object,int32,int32,int32,float32,float32,float32,float32,float32,float32,float32,category[known]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
%%time
out = df.groupby(df.passenger_count).trip_distance.mean().compute()

CPU times: user 37.7 s, sys: 3.91 s, total: 41.6 s
Wall time: 6min 28s


In [6]:
out

passenger_count
0      2.762565
1      2.877378
2      3.091487
3      3.041540
4      3.111632
5      3.012087
6      2.990773
7      3.275516
8      5.036861
9      5.982670
192    1.010000
96     0.830000
Name: trip_distance, dtype: float64

In [5]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:43161 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [7]:
df = dd.read_parquet("datasets/nyctaxi/processed/",
                    columns=["passenger_count", "trip_distance"])

In [8]:
%%time
out = df.groupby(df.passenger_count).trip_distance.mean().compute()

CPU times: user 4.57 s, sys: 353 ms, total: 4.92 s
Wall time: 15.8 s


In [9]:
out

passenger_count
0      2.762565
1      2.877378
2      3.091487
3      3.041540
4      3.111632
5      3.012087
6      2.990773
7      3.275516
8      5.036861
9      5.982670
192    1.010000
96     0.830000
Name: trip_distance, dtype: float64

# Clean data

In [1]:
import pandas as pd
import os
import dask.dataframe as dd
from dask import delayed, compute
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:40055 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [2]:
df = dd.read_parquet("datasets/nyctaxi/processed/",
                     columns=["fare_amount", "tip_amount",
                               "total_amount"])

In [3]:
%%time
out = df.describe(percentiles=[.05,.25,.5,.75, .95, .99])\
        .compute()

CPU times: user 26.3 s, sys: 2.05 s, total: 28.4 s
Wall time: 1min 6s


In [4]:
out

,fare_amount,tip_amount,total_amount
count,2.257315e+08,2.257315e+08,2.257315e+08
mean,1.303678e+01,1.855182e+00,1.637630e+01
std,1.702013e+02,2.616502e+00,1.704146e+02
min,-8.000000e+02,-3.910000e+02,-8.003000e+02
5%,4.500000e+00,0.000000e+00,6.300000e+00
25%,7.000000e+00,0.000000e+00,9.300000e+00
50%,1.050000e+01,1.560000e+00,1.330000e+01
75%,1.750000e+01,2.750000e+00,2.080000e+01
95%,4.200000e+01,7.410000e+00,5.258000e+01
99%,5.750000e+01,1.302000e+01,7.567000e+01


In [5]:
def clean(fn, fldr_in, fldr_out):
    fn_out =  fn.replace(fldr_in, fldr_out)
    df = pd.read_parquet(fn, use_threads=False)
    df = df[(df["passenger_count"]<=9) &
            (df["tpep_pickup_datetime"]<df["tpep_dropoff_datetime"]) &
            (df["fare_amount"].between(0, 200)) &
            (df["tip_amount"].between(0, 50)) &
            (df["total_amount"].between(0, 250))].reset_index(drop=True)
    df.to_parquet(fn_out, index=False)

In [6]:
jobs = []
fldr_in = "datasets/nyctaxi/processed/"
fldr_out = "datasets/nyctaxi/processed_clean/"
for (dirpath, dirnames, filenames) in os.walk(fldr_in):
    if len(filenames) > 0:
        jobs += [os.path.join(dirpath, fn)
                 for fn in filenames]

In [7]:
for year in [2017, 2018]:
    os.makedirs(os.path.join(fldr_out, f"year={year}"), exist_ok=True)

In [8]:
client.restart()

Client Scheduler: tcp://127.0.0.1:40055 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [9]:
%%time
to_process = [clean(job, fldr_in, fldr_out) for job in jobs]
out = compute(to_process)

CPU times: user 4min 7s, sys: 37.1 s, total: 4min 44s
Wall time: 4min 18s


# Some analytics

In [1]:
import pandas as pd
import os
import dask.dataframe as dd
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:34743 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


## Percentage tips my payment type

In [2]:
df = dd.read_parquet("datasets/nyctaxi/processed_clean/",
                     columns=["payment_type",
                              "tip_amount",
                              "total_amount"])

In [8]:
payment_dict = {1: 'Credit Card',
                2: 'Cash',
                3: 'No Charge',
                4: 'Dispute',
                5: 'Unknown',
                6: 'Voided trip'}

In [9]:
df["payment_name"] = df["payment_type"].map(payment_dict)

In [10]:
out = df.groupby("payment_name")[["tip_amount", "total_amount"]].sum()

In [11]:
%%time
out = out.compute()

CPU times: user 7.05 s, sys: 458 ms, total: 7.51 s
Wall time: 28.5 s


In [24]:
out["per"] = out["tip_amount"]/out["total_amount"]

In [12]:
out

,tip_amount,total_amount
payment_name,,
Cash,2.043670e+03,9.287791e+08
Credit Card,4.169920e+08,2.718722e+09
Dispute,1.241520e+03,4.734770e+06
No Charge,3.171420e+03,1.591826e+07
Unknown,0.000000e+00,5.110000e+01


In [13]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:34743 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [14]:
df = dd.read_parquet("datasets/nyctaxi/processed_clean/",
                     columns=["payment_type",
                              "tip_amount",
                              "total_amount"])

In [15]:
out = df.groupby("payment_type")[["tip_amount", "total_amount"]].sum()

In [16]:
%%time
out = out.compute()

CPU times: user 5.27 s, sys: 346 ms, total: 5.62 s
Wall time: 17.9 s


In [18]:
out["per"] = out["tip_amount"]/out["total_amount"]

In [20]:
out = out.reset_index()

In [21]:
out["payment_type"] = out["payment_type"].map(payment_dict)

In [22]:
out

,payment_type,tip_amount,total_amount,per
0,Credit Card,4.169920e+08,2.718722e+09,0.153378
1,Cash,2.043670e+03,9.287791e+08,0.000002
2,No Charge,3.171420e+03,1.591826e+07,0.000199
3,Dispute,1.241520e+03,4.734770e+06,0.000262
4,Unknown,0.000000e+00,5.110000e+01,0.000000


## trip duration

In [36]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:34743 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [37]:
df = dd.read_parquet("datasets/nyctaxi/processed_clean/",
                     columns=['tpep_pickup_datetime',
                              'tpep_dropoff_datetime'])

In [38]:
out = (df["tpep_dropoff_datetime"]-df["tpep_pickup_datetime"]).dt.seconds

In [39]:
%%time
out = out.describe().compute()

CPU times: user 12.5 s, sys: 908 ms, total: 13.4 s
Wall time: 30.7 s


In [35]:
out

count    2.253626e+08
mean     1.020477e+03
std      3.709174e+03
min      1.000000e+00
25%      4.650000e+02
50%      8.100000e+02
75%      1.368000e+03
max      8.639900e+04
dtype: float64

In [41]:
1e3/60

16.666666666666668

In [42]:
8.1e2/60

13.5

## Does people tip more during holidays/weekends?

In [110]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:34743 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.52 GB


In [111]:
import holidays

In [112]:
NYholidays = holidays.UnitedStates(years=[2017, 2018], state='NY')

In [113]:
NYholidays = [a for a  in NYholidays.keys()]

In [114]:
df = dd.read_parquet("datasets/nyctaxi/processed_clean/",
                     columns=["tpep_pickup_datetime",
                              "payment_type",
                              "tip_amount",
                              "total_amount"])

In [115]:
df["is_holiday"] = ((df["tpep_pickup_datetime"].dt.weekday>=5) |
                    (df["tpep_pickup_datetime"].dt.date.isin(NYholidays))).astype("int32")

In [116]:
out = df.groupby(["payment_type", "is_holiday"])[["tip_amount", "total_amount"]].sum()

In [117]:
%%time
out = out.compute()

CPU times: user 21.9 s, sys: 1.45 s, total: 23.3 s
Wall time: 1min 47s


In [118]:
out["per"] = out["tip_amount"]/out["total_amount"]

In [119]:
out = out.reset_index()

In [120]:
out["payment_type"] = out["payment_type"].map(payment_dict)

In [121]:
out

,payment_type,is_holiday,tip_amount,total_amount,per
0,Credit Card,0,2.999732e+08,1.948192e+09,0.153975
1,Credit Card,1,1.170188e+08,7.705299e+08,0.151868
2,Cash,0,1.490670e+03,6.270033e+08,0.000002
3,Cash,1,5.530000e+02,3.017757e+08,0.000002
4,No Charge,0,1.983500e+03,1.089151e+07,0.000182
5,No Charge,1,1.187920e+03,5.026750e+06,0.000236
6,Dispute,0,7.627000e+02,3.166080e+06,0.000241
7,Dispute,1,4.788200e+02,1.568690e+06,0.000305
8,Unknown,1,0.000000e+00,1.000000e+01,0.000000
9,Unknown,0,0.000000e+00,4.110000e+01,0.000000
